In [21]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
import torch
import os

In [22]:
dataset_scores = pd.read_pickle('dataset_scores.pkl')
fr_original = pd.read_pickle('fr_original.pkl')
nr_metrics = pd.read_pickle('nr_metrics.pkl')

In [39]:
np.sort(pd.unique(nr_metrics.index.get_level_values('Video')))

array(['Dianying_1.mp4', 'Dianying_2.mp4', 'Dianying_3.mp4',
       'Fengjing1_1.mp4', 'Fengjing1_2.mp4', 'Fengjing1_3.mp4',
       'Fengjing3_1.mp4', 'Fengjing3_2.mp4', 'Fengjing3_3.mp4',
       'G1AbandonedKingdom_ERP_7680x3840_fps30_qp27_45406k.mp4',
       'G1AbandonedKingdom_ERP_7680x3840_fps30_qp37_9283k.mp4',
       'G1AbandonedKingdom_ERP_7680x3840_fps30_qp42_4140k.mp4',
       'G1Aerial_ERP_7680x3840_fps25_qp27_18646k.mp4',
       'G1Aerial_ERP_7680x3840_fps25_qp37_3307k.mp4',
       'G1Aerial_ERP_7680x3840_fps25_qp42_1216k.mp4',
       'G1BajaCalifonia_ERP_3840x2160_fps23.976_qp27_41186k.mp4',
       'G1BajaCalifonia_ERP_3840x2160_fps23.976_qp37_12633k.mp4',
       'G1BajaCalifonia_ERP_3840x2160_fps23.976_qp42_5570k.mp4',
       'G1BikingToWork_ERP_3840x2160_fps23.976_qp27_12306k.mp4',
       'G1BikingToWork_ERP_3840x2160_fps23.976_qp37_3596k.mp4',
       'G1BikingToWork_ERP_3840x2160_fps23.976_qp42_1855k.mp4',
       'G1LandSalt_ERP_6144x3072_fps29.97_qp27_9229k.mp4',
      

In [41]:
np.sort(pd.unique(dataset_scores.index))

array(['G10BoatInPark_4096x2048_fps30.mp4',
       'G10BoatInPark_ERP_4096x2048_fps30_qp27_14547k.mp4',
       'G10BoatInPark_ERP_4096x2048_fps30_qp37_3270k.mp4',
       'G10BoatInPark_ERP_4096x2048_fps30_qp42_1507k.mp4',
       'G10BoatInPark_RCMP_3072x2048_fps30_qp27_11884k.mp4',
       'G10BoatInPark_RCMP_3072x2048_fps30_qp37_2514k.mp4',
       'G10BoatInPark_RCMP_3072x2048_fps30_qp42_1095k.mp4',
       'G10BoatInPark_TSP_4096x2048_fps30_qp27_12477k.mp4',
       'G10BoatInPark_TSP_4096x2048_fps30_qp37_2592k.mp4',
       'G10BoatInPark_TSP_4096x2048_fps30_qp42_1172k.mp4',
       'G10BodybuildingWorkout_7680x3840_fps29.97.mp4',
       'G10BodybuildingWorkout_ERP_7680x3840_fps29.97_qp27_6105k.mp4',
       'G10BodybuildingWorkout_ERP_7680x3840_fps29.97_qp37_913k.mp4',
       'G10BodybuildingWorkout_ERP_7680x3840_fps29.97_qp42_697k.mp4',
       'G10BodybuildingWorkout_RCMP_5760x3840_fps29.97_qp27_6488k.mp4',
       'G10BodybuildingWorkout_RCMP_5760x3840_fps29.97_qp37_1036k.mp4',
       '

In [42]:
nr_metrics.head(10)

Blockiness:       SA:   Letterbox:   Pillarbox:   \
Video              Frame:                                                      
Yanchanghui2_2.mp4 0             0.87214  63.81669          0.0          0.0   
                   1             0.87400  63.82003          0.0          0.0   
                   2             0.87334  63.82198          0.0          0.0   
                   3             0.87358  63.82464          0.0          0.0   
                   4             0.87424  63.82559          0.0          0.0   
                   5             0.87446  63.82530          0.0          0.0   
                   6             0.87423  63.84066          0.0          0.0   
                   7             0.87474  63.83789          0.0          0.0   
                   8             0.87563  63.83931          0.0          0.0   
                   9             0.87561  63.84070          0.0          0.0   

                            Blockloss:   Blur:      TA:   Blackout:   \
Video              Frame:                                              
Yanchanghui2_2.mp4 0            1.71875   5.318  0.00000           0   
                   1            1.81250   5.304  1.22376           0   
                   2            1.96875   5.303  1.07490           0   
                   3            1.91406   5.302  0.63068           0   
                   4            1.78906   5.298  1.95317           0   
                   5            1.79688   5.295  1.33072           0   
                   6            1.87500   5.301  1.37948           0   
                   7            1.77344   5.297  1.50980           0   
                   8            1.81250   5.295  1.40829           0   
                   9            1.81250   5.295  0.47406           0   

                            Freezing:   Exposure(bri):   Contrast:   \
Video              Frame:                                             
Yanchanghui2_2.mp4 0                 0              125    54.68518   
                   1                 0              125    54.68402   
                   2                 0              125    54.68221   
                   3                 0              125    54.68333   
                   4                 0              125    54.68087   
                   5                 0              125    54.68509   
                   6                 0              125    54.68566   
                   7                 0              125    54.68788   
                   8                 0              125    54.68972   
                   9                 0              125    54.68995   

                            Interlace:   Noise:  Slice:   Flickering:  
Video              Frame:                                              
Yanchanghui2_2.mp4 0            0.00593   0.3841  46.292      0.00000  
                   1            0.00593   0.3841  46.292     -1.00000  
                   2            0.00595   0.3841  46.292     -1.00000  
                   3            0.00594   0.3841  46.292     -1.00000  
                   4            0.00594   0.3841  46.292     -1.00000  
                   5            0.00592   0.3841  46.292     -1.00000  
                   6            0.00591   0.3841  46.292     -1.00000  
                   7            0.00591   0.3841  46.292     -1.00000  
                   8            0.00590   0.3841  46.292      0.42803  
                   9            0.00590   0.3841  46.292     -1.00000

In [43]:
dataset_scores.head(10)

,Raw,DMOS
Video,,
G8ANewEmpire_3840x2048_fps29.97.mp4,82.849294,0.000000
G8ANewEmpire_ERP_3840x2048_fps29.97_qp27_33327k.mp4,80.023548,43.448893
G8ANewEmpire_ERP_3840x2048_fps29.97_qp37_10885k.mp4,72.880691,51.821361
G8ANewEmpire_ERP_3840x2048_fps29.97_qp42_5571k.mp4,73.979592,50.533289
G8ANewEmpire_RCMP_2880x1920_fps29.97_qp27_31147k.mp4,78.532182,45.196991
G8ANewEmpire_RCMP_2880x1920_fps29.97_qp37_8905k.mp4,83.477237,39.400667
G8ANewEmpire_RCMP_2880x1920_fps29.97_qp42_4354k.mp4,67.072214,58.629741
G8ANewEmpire_TSP_3840x1920_fps29.97_qp27_31798k.mp4,73.665620,50.901310
G8ANewEmpire_TSP_3840x1920_fps29.97_qp37_9645k.mp4,73.194662,51.453340


In [56]:
nr_metrics.describe()

,Blockiness:,SA:,Letterbox:,Pillarbox:,Blockloss:,Blur:,TA:,Blackout:,Freezing:,Exposure(bri):,Contrast:,Interlace:,Noise:,Flickering:
count,70455.000000,70455.000000,70455.000000,70455.000000,70455.000000,70455.000000,70455.000000,70455.0,70455.000000,70455.000000,70455.000000,70455.000000,70455.000000,70455.000000
mean,0.910629,49.279873,0.001279,0.000092,4.115915,5.333406,10.127091,0.0,0.001135,125.500092,63.126858,0.002258,0.393142,-0.794053
std,0.063610,25.061043,0.011090,0.001638,5.350096,1.664809,7.706963,0.0,0.033678,5.316771,30.349890,0.002604,0.338158,0.544583
min,0.414150,4.433490,0.000000,0.000000,0.000000,2.264000,0.000000,0.0,0.000000,68.000000,2.655120,0.000000,0.000000,-1.000000
25%,0.883795,30.556580,0.000000,0.000000,0.868060,4.041000,3.529010,0.0,0.000000,123.000000,44.062260,0.000840,0.251100,-1.000000
50%,0.925900,45.155780,0.000000,0.000000,2.183330,5.089000,8.266490,0.0,0.000000,126.000000,53.318500,0.001510,0.300800,-1.000000
75%,0.954440,63.643510,0.000000,0.000000,5.095560,6.263000,15.541630,0.0,0.000000,127.000000,73.117325,0.002570,0.393700,-1.000000
max,1.017000,133.286830,0.234380,0.102540,67.788890,14.177000,75.560340,0.0,1.000000,145.000000,142.082680,0.020940,3.999700,0.931360


In [63]:
nr_metrics.loc[nr_metrics['Flickering:'] != -1]

Blockiness:       SA:   Letterbox:   Pillarbox:   \
Video              Frame:                                                      
Yanchanghui2_2.mp4 0             0.87214  63.81669      0.00000      0.00000   
                   8             0.87563  63.83931      0.00000      0.00000   
                   16            0.87337  63.81988      0.00000      0.00000   
                   24            0.87281  63.58878      0.00000      0.00000   
                   32            0.87386  63.45976      0.00000      0.00000   
                   40            0.87481  63.56624      0.00000      0.00000   
                   48            0.87599  63.80162      0.00000      0.00000   
                   56            0.87596  63.90932      0.00000      0.00000   
                   64            0.87759  63.86491      0.00000      0.00000   
                   72            0.88298  63.80073      0.00000      0.00000   
                   80            0.88900  64.04587      0.00000      0.00000   
                   88            0.89170  64.18984      0.00000      0.00000   
                   96            0.89765  64.27077      0.00000      0.00000   
                   104           0.89995  64.34592      0.00000      0.00000   
                   112           0.90286  64.43810      0.00000      0.00000   
                   120           0.90334  64.48023      0.00000      0.00000   
                   128           0.90501  64.56027      0.00000      0.00000   
                   136           0.90640  63.95477      0.00000      0.00000   
                   144           0.90699  64.24991      0.00000      0.00000   
                   152           0.90546  64.23552      0.00000      0.00000   
                   160           0.90623  64.30954      0.00000      0.00000   
                   168           0.90796  64.33503      0.00000      0.00000   
                   176           0.91113  64.38274      0.00000      0.00000   
                   184           0.91540  64.57461      0.00000      0.00000   
                   192           0.91491  65.36152      0.00000      0.00000   
                   200           0.91306  65.22436      0.00000      0.00000   
                   208           0.91550  65.40861      0.00000      0.00000   
                   216           0.91507  65.56538      0.00000      0.00000   
                   224           0.91333  65.60582      0.00000      0.00000   
                   232           0.91468  65.26631      0.00000      0.00000   
...                                  ...       ...          ...          ...   
Yanchanghui1_3.mp4 64            0.81396  18.59459      0.00000      0.00000   
                   72            0.81794  17.41571      0.00000      0.00684   
                   80            0.84799  12.50213      0.00000      0.01270   
                   88            0.81216  14.07847      0.00000      0.00000   
                   96            0.83873  17.43141      0.00000      0.00000   
                   104           0.81324   8.90695      0.00000      0.02979   
                   112           0.88625  18.06820      0.00000      0.00635   
                   120           0.89383  18.72959      0.00000      0.00000   
                   128           0.85661  17.80465      0.00977      0.00000   
                   136           0.91109  23.20337      0.00684      0.00000   
                   144           0.89130  24.36278      0.00586      0.00000   
                   152           0.73248  11.34967      0.03809      0.00000   
                   160           0.64782   7.56086      0.03809      0.00000   
                   168           0.72274   7.83674      0.03809      0.00537   
                   176           0.81047   7.01568      0.03809      0.00537   
                   184           0.77205   9.08271      0.03809      0.00537   
                   192           0.90530  23.07996      0.02148      0.00000   
                   200      